# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
PATH = "data/demand-forecasting/"

# Helper Functions

In [ ]:
def add_datepart(df, fldname, drop=True, time=False):
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [ ]:
# custom evaluation metric
def SMAPE(y_pred, dtrain):
    y_true = dtrain.get_label()
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 'SMAPE', np.nanmean(diff)

In [ ]:
def smape2(y_pred, y_true):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff)

In [ ]:
def print_scores(pred,label):
    rmse = np.sqrt(mean_squared_error(pred,label))
    mae = mean_absolute_error(pred,label)
    smape_score = smape2(pred,label)
    
    print('RMSE\t\t ' + str(rmse))
    print('SMAPE\t\t' + str(smape_score))

# Pre-Process Data

In [ ]:
train = pd.read_csv(f'{PATH}train.csv', parse_dates=['date'])
test = pd.read_csv(f'{PATH}test.csv', parse_dates=['date'], index_col='id')

In [10]:
joined = pd.concat([train,test], sort=False)

In [11]:
add_datepart(joined, 'date')

In [12]:
joined["median-store_item-month"] = joined.groupby(['Month',"item","store"])["sales"].transform("median") # median sales for particular item-store month combo
joined["mean-store_item-week"] = joined.groupby(['Week',"item","store"])["sales"].transform("mean") # mean sales for particular item-store week combo

joined["item-month-sum"] = joined.groupby(['Month',"item"])["sales"].transform("sum") # total sales of that item  for all stores
joined["store-month-sum"] = joined.groupby(['Month',"store"])["sales"].transform("sum") # total sales of that store  for all items

In [13]:
joined.drop(columns=['Is_month_end', 'Is_month_start','Is_quarter_end',
                 'Is_quarter_start','Is_year_end','Is_year_start'],inplace=True)

In [14]:
train = joined[~joined['sales'].isna()]
test = joined[joined['sales'].isna()]

sales = train.pop('sales')
test = test.drop('sales', axis=1)

In [15]:
display(train.head(2))
display(pd.DataFrame(sales).head(2))
display(test.head(2))

,store,item,Year,Month,Week,Day,Dayofweek,Dayofyear,Elapsed,median-store_item-month,mean-store_item-week,item-month-sum,store-month-sum
0,1,1,2013,1,1,1,1,1,1356998400,13.0,13.970588,22987.0,249352.0
1,1,1,2013,1,1,2,2,2,1357084800,13.0,13.970588,22987.0,249352.0


,sales
0,13.0
1,11.0


,store,item,Year,Month,Week,Day,Dayofweek,Dayofyear,Elapsed,median-store_item-month,mean-store_item-week,item-month-sum,store-month-sum
0,1,1,2018,1,1,1,0,1,1514764800,13.0,13.970588,22987.0,249352.0
1,1,1,2018,1,1,2,1,2,1514851200,13.0,13.970588,22987.0,249352.0


# Split Training-Validation Data

In [16]:
train.shape, sales.shape, test.shape

((913000, 13), (913000,), (45000, 13))

In [17]:
# closest same period (diff year) as test set
X_val = train.loc[(train.Year==2017) & ((train.Month==10) | (train.Month==11) | (train.Month==12))].copy() 
y_val = sales[X_val.index].copy()

X_train = train.drop(X_val.index).copy()
y_train = sales.drop(X_val.index).copy()

In [18]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((867000, 13), (867000,), (46000, 13), (46000,))

In [19]:
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_val.reset_index(drop=True, inplace=True)
y_val.reset_index(drop=True, inplace=True)
print()

# XGBoost

In [20]:
DM_train = xgb.DMatrix(data=X_train, label=y_train)
DM_val = xgb.DMatrix(data=X_val, label=y_val)
DM_test = xgb.DMatrix(data=test)

In [21]:
evals_result = {}
watchlist = [(DM_train, "training"), (DM_val, "validation")]

In [25]:
params_native = {
    'objective': 'reg:linear', 
    'booster':'gbtree',        
    'silent': 1,               
    'eta': 0.005,                 
    'gamma': 0,                 
    'max-depth': 8,             
    'min_child_weight': 0.9,      
    'max_delta_step': 0,        
    'subsample': 0.8,             
    'colsample_bytree': 0.7,      
    'colsample_bylevel': 0.7,     
    'lambda': 0.9,                
    'alpha': 0,                 
    'scale_pos_weight': 1,      
    'base_score': 0.5,          
    'eval_metric':'rmse',      
    'seed': 42                  
}

## Train with partial data

In [26]:
%%time
xgb_native = xgb.train(params=params_native, 
                            dtrain=DM_train,
                            num_boost_round=100_000,
                            evals=watchlist,
                            early_stopping_rounds=20,
                            evals_result=evals_result,
                            feval=SMAPE,
                            maximize=False,
                            verbose_eval=50)

[0]	training-rmse:58.8333	validation-rmse:60.932	training-SMAPE:192.607	validation-SMAPE:193.173
Multiple eval metrics have been passed: 'validation-SMAPE' will be used for early stopping.

Will train until validation-SMAPE hasn't improved in 20 rounds.
[50]	training-rmse:46.1902	validation-rmse:48.4675	training-SMAPE:121.437	validation-SMAPE:125.588
[100]	training-rmse:36.4103	validation-rmse:38.7638	training-SMAPE:82.0401	validation-SMAPE:87.0794
[150]	training-rmse:28.8631	validation-rmse:31.1361	training-SMAPE:57.7474	validation-SMAPE:62.6342
[200]	training-rmse:23.0815	validation-rmse:25.2272	training-SMAPE:41.8913	validation-SMAPE:46.2713
[250]	training-rmse:18.6514	validation-rmse:20.6433	training-SMAPE:31.4386	validation-SMAPE:35.1792
[300]	training-rmse:15.3094	validation-rmse:17.1186	training-SMAPE:24.5261	validation-SMAPE:27.5518
[350]	training-rmse:12.8352	validation-rmse:14.454	training-SMAPE:20.0181	validation-SMAPE:22.3495
[400]	training-rmse:11.0368	validation-rmse:12.4

In [24]:
print("Best iteration\t{}".format(xgb_native.best_iteration))
print("Best tree limit\t{}".format(xgb_native.best_ntree_limit))
print("Best RMSE score\t{}".format(xgb_native.best_score))

print("\nValidation score")
pred_val = xgb_native.predict(DM_val)
print_scores(pred_val, y_val)

print("\nTraining score")
pred_train = xgb_native.predict(DM_train)
print_scores(pred_train, y_train)

Best iteration	1532
Best tree limit	1533
Best RMSE score	12.120908

Validation score
RMSE		 7.461529418803279
SMAPE		12.121071807344874

Training score
RMSE		 7.116308456081811
SMAPE		12.36981618359407


## Predict

In [ ]:
# y_pred = xgb_native.predict(DM_test, ntree_limit = xgb_native.best_ntree_limit)

# Train with full data based on tuned parameters

In [ ]:
%%time
full_data = xgb.DMatrix(data=train, label=sales)
xgb_native_full_data = xgb.train(params=params_native, 
                                dtrain=full_data,
                                num_boost_round=1447,
                                evals=[(full_data, "full data training")],
                                feval=SMAPE,
                                maximize=False,
                                verbose_eval=100)

## Predict

In [ ]:
y_pred = xgb_native_full_data.predict(DM_test)

# Submit

In [ ]:
submission = pd.read_csv(f'{PATH}test.csv', index_col='id')

In [ ]:
submission['sales'] = y_pred

In [ ]:
csv_fn = f'{PATH}tmp/XGB_v5_1.csv'

In [ ]:
submission[['sales']].to_csv(csv_fn)

# Plot the feature importances

In [ ]:
xgb_native_full_data.save_model(f'{PATH}tmp/XGB_v4_partial_4')

In [ ]:
xgb.plot_importance(xgb_native_full_data)
plt.show()